Welcome to this notebook where we'll be implementing a simple RNN character model with PyTorch to familiarize ourselves with the PyTorch library and get started with RNNs. You can run the code we’re using on FloydHub by clicking the button below and creating the project as well.

[![Run on FloydHub](https://static.floydhub.com/button/button-small.svg)](https://floydhub.com/run?template=https://github.com/gabrielloye/RNN-walkthrough)

In this implementation, we'll be building a model that can complete your sentence based on a few characters or a word used as input.
![Example](img/Slide4.jpg)
To keep this short and simple, we won't be using any large or external datasets. Instead, we'll just be defining a few sentences to see how the model learns from these sentences. The process that this implementation will take is as follows:
![Overview](img/Slide5.jpg)

We'll start off by importing the main PyTorch package along with the *Variable* class used to store our data tensors and the *nn* package which we will use when building the model. In addition, we'll only be using numpy to pre-process our data as Torch works really well with numpy.

In [199]:
import torch
from torch import nn
import pandas as pd
import json
import numpy as np

First, we'll define the sentences that we want our model to output when fed with the first word or the first few characters.

Then we'll create a dictionary out of all the characters that we have in the sentences and map them to an integer. This will allow us to convert our input characters to their respective integers (*char2int*) and vice versa (*int2char*).

In [200]:
text3 = ['hey how are you','good i am fine','have a nice day']
df = pd.read_csv('../sentences.csv')
text = df['input'].tolist()
text2 = df['target'].tolist()
input_seq = df['input'].tolist()
target_seq = df['target'].tolist()
text = text + text2
print(text)
print(text3)

# Join all the sentences together and extract the unique characters from the combined sentences
#chars = set(''.join(text))
chars = set(''.join(text))

# Creating a dictionary that maps integers to the characters
int2char = dict(enumerate(chars))
#int2char = dict(enumerate(chars))

# Creating another dictionary that maps characters to integers
char2int = {char: ind for ind, char in int2char.items()}

['terrain ujwo with position 74980 and amount 1173 to 9892 and class(s) owjk and position in jjlih and material(s) dicgxwdt and rotation 469-20-892 and amount 3885 to 7331 ', 'skylight wvdcby with intensity 542 lux as well as object lklez with rotation 3-4-4 to 860-0-7  ', 'object ymus with visibility false  as well as skylight pcjg with rotation 02-08-8 ', 'skylight sezkcghs with intensity 800 lux as well as terrain gkur with rotation 36-93-875 to 139-059-970 and material(s) rnvxtwif-fepf and material(s) aubkfiuti-ghbs ', 'skylight wevr with intensity 113 lux ', 'object rsag with rotation 302-9-02 to 785-9-896  ', 'terrain hvdjv with rotation 1-5-975 to 426-792-67 ', 'object eeeogbzpz with rotation 04-6-670 and overlap false and extent 1-969-15 and amount 7994 to 3020 and position 1048715 and scale 8 and visibility false and extent 928-588-5 and material(s) yllhovito-lfrwi and rotation 532-6-4 to 75-1-78 and class(s) ompqclqf-slaep and amount 7353 to 4489 and visibility false and clas

In [201]:
print(char2int)

{'u': 0, 'g': 1, '6': 2, 'm': 3, ':': 4, 'a': 5, ']': 6, 'r': 7, '3': 8, 'c': 9, 'q': 10, 'p': 11, '{': 12, '4': 13, '5': 14, 'x': 15, '9': 16, '/': 17, 'z': 18, 'e': 19, 't': 20, 'b': 21, '-': 22, 'y': 23, '_': 24, 'd': 25, ' ': 26, '8': 27, '2': 28, 'h': 29, '7': 30, '0': 31, 'w': 32, 'l': 33, 'n': 34, '}': 35, 's': 36, '[': 37, 'j': 38, 'i': 39, '(': 40, '\\': 41, ')': 42, '1': 43, 'f': 44, 'k': 45, 'o': 46, 'v': 47}


Next, we'll be padding our input sentences to ensure that all the sentences are of the sample length. While RNNs are typically able to take in variably sized inputs, we will usually want to feed training data in batches to speed up the training process. In order to used batches to train on our data, we'll need to ensure that each sequence within the input data are of equal size.

Therefore, in most cases, padding can be done by filling up sequences that are too short with **0** values and trimming sequences that are too long. In our case, we'll be finding the length of the longest sequence and padding the rest of the sentences with blank spaces to match that length.

In [202]:
maxlen = len(max(text, key=len))
print("The longest string has {} characters".format(maxlen))

The longest string has 1344 characters


In [203]:
# Padding

# A simple loop that loops through the list of sentences and adds a ' ' whitespace until the length of the sentence matches
# the length of the longest sentence
half = len(text) - len(input_seq)
print(half)
for i in range(half):
    while len(input_seq[i])<maxlen:
        input_seq[i] += ' '
    while len(target_seq[i])<maxlen:
        target_seq[i] += ' '


100


As we're going to predict the next character in the sequence at each time step, we'll have to divide each sentence into

- Input data
    - The last input character should be excluded as it does not need to be fed into the model
- Target/Ground Truth Label
    - One time-step ahead of the Input data as this will be the "correct answer" for the model at each time step corresponding to the input data

In [204]:
# Creating lists that will hold our input and target sequences
# input_seq = []
# target_seq = []

# for i in range(len(text)):
#     # Remove last character for input sequence
#     input_seq.append(text[i][:-1])
    
#     # Remove firsts character for target sequence
#     target_seq.append(text[i][1:])
#     print("Input Sequence: {}\nTarget Sequence: {}".format(input_seq[i], target_seq[i]))


Now we can convert our input and target sequences to sequences of integers instead of characters by mapping them using the dictionaries we created above. This will allow us to one-hot-encode our input sequence subsequently.

In [205]:
for i in range(half):
        input_seq[i] = [char2int[character] for character in input_seq[i]]
        target_seq[i] = [char2int[character] for character in target_seq[i]]

Before encoding our input sequence into one-hot vectors, we'll define 3 key variables:

- *dict_size*: The number of unique characters that we have in our text
    - This will determine the one-hot vector size as each character will have an assigned index in that vector
- *seq_len*: The length of the sequences that we're feeding into the model
    - As we standardised the length of all our sentences to be equal to the longest sentences, this value will be the max length - 1 as we removed the last character input as well
- *batch_size*: The number of sentences that we defined and are going to feed into the model as a batch

In [206]:
dict_size = len(char2int)
seq_len = maxlen-1
batch_size = half

def one_hot_encode(sequence, dict_size, seq_len, batch_size):
    # Creating a multi-dimensional array of zeros with the desired output shape
    features = np.zeros((batch_size, seq_len, dict_size), dtype=np.float32)
    
    # Replacing the 0 at the relevant character index with a 1 to represent that character
    for i in range(batch_size):
        for u in range(seq_len):
            features[i, u, sequence[i][u]] = 1
    return features

We also defined a helper function that creates arrays of zeros for each character and replaces the corresponding character index with a **1**.

In [207]:
input_seq = one_hot_encode(input_seq, dict_size, seq_len, half)
target_seq = one_hot_encode(target_seq, dict_size, seq_len, half)

#print("Input shape: {} --> (Batch Size, Sequence Length, One-Hot Encoding Size)".format(input_seq.shape))

Since we're done with all the data pre-processing, we can now move the data from numpy arrays to PyTorch's very own data structure - **Torch Tensors**

In [208]:
input_seq = torch.from_numpy(input_seq)
target_seq = torch.Tensor(target_seq)

Now we've reached the fun part of this project! We'll be defining the model using the Torch library, and this is where you can add or remove layers, be it fully connected layers, convolutational layers, vanilla RNN layers, LSTM layers, and many more! In this post, we'll be using the basic nn.rnn to demonstrate a simple example of how RNNs can be used.

Before we start building the model, let's use a build in feature in PyTorch to check the device we're running on (CPU or GPU). This implementation will not require GPU as the training is really simple. However, as you progress on to large datasets and models with millions of trainable parameters, using the GPU will be very important to speed up your training.

In [209]:
# torch.cuda.is_available() checks and returns a Boolean True if a GPU is available, else it'll return False
is_cuda = torch.cuda.is_available()

# If we have a GPU available, we'll set our device to GPU. We'll use this device variable later in our code.
if is_cuda:
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU not available, CPU used")

GPU not available, CPU used


To start building our own neural network model, we can define a class that inherits PyTorch’s base class (nn.module) for all neural network modules. After doing so, we can start defining some variables and also the layers for our model under the constructor. For this model, we’ll only be using 1 layer of RNN followed by a fully connected layer. The fully connected layer will be in-charge of converting the RNN output to our desired output shape.

We’ll also have to define the forward pass function under forward() as a class method. The order the forward function is sequentially executed, therefore we’ll have to pass the inputs and the zero-initialized hidden state through the RNN layer first, before passing the RNN outputs to the fully-connected layer. Note that we are using the layers that we defined in the constructor.

The last method that we have to define is the method that we called earlier to initialize the hidden state - init_hidden(). This basically creates a tensor of zeros in the shape of our hidden states.

In [210]:
class Model(nn.Module):
    def __init__(self, input_size, output_size, hidden_dim, n_layers):
        super(Model, self).__init__()

        # Defining some parameters
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers

        #Defining the layers
        # RNN Layer
        self.rnn = nn.RNN(input_size, hidden_dim, n_layers, batch_first=True, nonlinearity='relu')   
        # Fully connected layer
        self.fc = nn.Linear(hidden_dim, output_size)
        
    
    def forward(self, x):
        
        batch_size = x.size(0)

        #Initializing hidden state for first input using method defined below
        hidden = self.init_hidden(batch_size)

        # Passing in the input and hidden state into the model and obtaining outputs
        out, hidden = self.rnn(x, hidden)
        

        # Reshaping the outputs such that it can be fit into the fully connected layer
        out = out.contiguous().view(-1, self.hidden_dim)
        out = self.fc(out)
        # Reshaping the outputs such that it can be fit into the fully connected layer
        #out = out.contiguous().view(-1, self.hidden_dim)
        #print("out 1",out.shape)
        #out = self.fc(out)
        #print("out 2", out.shape)
        #print("hidden", hidden.shape)
        return out, hidden
    
    def init_hidden(self, batch_size):
        # This method generates the first hidden state of zeros which we'll use in the forward pass
        hidden = torch.zeros(self.n_layers, batch_size, self.hidden_dim).to(device)
         # We'll send the tensor holding the hidden state to the device we specified earlier as well
        return hidden

After defining the model above, we'll have to instantiate the model with the relevant parameters and define our hyperparamters as well. The hyperparameters we're defining below are:

- *n_epochs*: Number of Epochs --> This refers to the number of times our model will go through the entire training dataset
- *lr*: Learning Rate --> This affects the rate at which our model updates the weights in the cells each time backpropogation is done
    - A smaller learning rate means that the model changes the values of the weight with a smaller magnitude
    - A larger learning rate means that the weights are updated to a larger extent for each time step

Similar to other neural networks, we have to define the optimizer and loss function as well. We’ll be using CrossEntropyLoss as the final output is basically a classification task.

In [217]:
# Instantiate the model with hyperparameters

model = Model(input_size=dict_size, output_size=dict_size, hidden_dim=30, n_layers=2)
# We'll also set the model to the device that we defined earlier (default is CPU)
model = model.to(device)

# Define hyperparameters
n_epochs = 300
lr=0.01

# Define Loss, Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

Now we can begin our training! As we only have a few sentences, this training process is very fast. However, as we progress, larger datasets and deeper models mean that the input data is much larger and the number of parameters within the model that we have to compute is much more.

In [218]:
# Training Run
input_seq = input_seq.to(device)
for epoch in range(1, n_epochs + 1):
    optimizer.zero_grad() # Clears existing gradients from previous epoch
    #input_seq = input_seq.to(device)
    output, hidden = model(input_seq)
    output = output.to(device)
    target_seq = target_seq.to(device)
    # print("output before",output.shape)
    # print("targ before", target_seq.shape)
    # print("targ", target_seq.view(-1).long().shape)
    targ = target_seq.reshape(target_seq.size(0) * target_seq.size(1), -1)
    # print("targ reshape", targ.shape)
    loss = criterion(output, targ)
    loss.backward() # Does backpropagation and calculates gradients
    optimizer.step() # Updates the weights accordingly
    
    if epoch%10 == 0:
        print('Epoch: {}/{}.............'.format(epoch, n_epochs), end=' ')
        print("Loss: {:.4f}".format(loss.item()))

Epoch: 10/300............. Loss: 0.6909
Epoch: 20/300............. Loss: 0.4974
Epoch: 30/300............. Loss: 0.4257
Epoch: 40/300............. Loss: 0.4090
Epoch: 50/300............. Loss: 0.4026
Epoch: 60/300............. Loss: 0.3987
Epoch: 70/300............. Loss: 0.3945
Epoch: 80/300............. Loss: 0.3970
Epoch: 90/300............. Loss: 0.3909
Epoch: 100/300............. Loss: 0.3860
Epoch: 110/300............. Loss: 0.3817
Epoch: 120/300............. Loss: 0.3770
Epoch: 130/300............. Loss: 0.3867
Epoch: 140/300............. Loss: 0.3817
Epoch: 150/300............. Loss: 0.3782
Epoch: 160/300............. Loss: 0.3747
Epoch: 170/300............. Loss: 0.3715
Epoch: 180/300............. Loss: 0.3687
Epoch: 190/300............. Loss: 0.3659
Epoch: 200/300............. Loss: 0.3733
Epoch: 210/300............. Loss: 0.3625
Epoch: 220/300............. Loss: 0.4946
Epoch: 230/300............. Loss: 0.4370
Epoch: 240/300............. Loss: 0.4028
Epoch: 250/300...........

In [267]:
import torch
import torch.nn as nn
from gensim.models import Word2Vec

# define the RNN model
class RNNModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers):
        super(RNNModel, self).__init__()
        
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.rnn = nn.RNN(hidden_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        batch_size = x.size(0)
        hidden = self.init_hidden(batch_size)
        x = x.long()
        embedded = self.embedding(x)
        out, hidden = self.rnn(embedded, hidden)
        out = out.contiguous().view(-1, self.hidden_size)
        out = self.fc(out)
        
        return out, hidden
    
    def init_hidden(self, batch_size):
        return torch.zeros(self.num_layers, batch_size, self.hidden_size)

# define the Word2Vec model
class W2VModel():
    def __init__(self, sentences):
        self.model = Word2Vec(sentences, min_count=1)

    def encode(self, sentence):
        vecs = []
        sentence2 = str(sentence)
        sentences3 = sentence2.split()
        for word in sentences3:
            if word in self.model.wv.key_to_index:
                vecs.append(self.model[word])
        return torch.Tensor(vecs)

# define the training loop


In [286]:
def train(model, w2v_model, optimizer, criterion, train_loader, num_epochs):
    for epoch in range(num_epochs):
        for i, (inputs, targets) in enumerate(train_loader):
            optimizer.zero_grad()
            print(inputs)
            encoded_inputs = w2v_model.encode(inputs)
            encoded_targets = w2v_model.encode(targets)
            print("in",encoded_inputs)
            print("out",encoded_targets)
            
            outputs, _ = model(encoded_inputs.unsqueeze(0))
            
            loss = criterion(outputs, encoded_targets)
            loss.backward()
            optimizer.step()
            
            if (i+1) % 10 == 0:
                print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                       .format(epoch+1, num_epochs, i+1, len(train_loader), loss.item()))

In [294]:

# sample data
sentences = ['this is a sample sentence.', 'this is another sample sentence.']
inputs = ['this is the input.', 'this is another input.']
targets = ['this is the target output.', 'this is another target output.']
allStuff = inputs + targets
print("inputs and targs", allStuff)
# create the Word2Vec model
w2v_model = W2VModel(allStuff)
print("w2v", w2v_model.model)


inputs and targs ['this is the input.', 'this is another input.', 'this is the target output.', 'this is another target output.']
w2v Word2Vec(vocab=14, vector_size=100, alpha=0.025)


In [295]:
input_size = len(w2v_model.model.wv)
print(input_size)
hidden_size = 100
output_size = 100
num_layers = 1
model = RNNModel(input_size, hidden_size, output_size, num_layers)

# define the optimizer and loss function
learning_rate = 0.01
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

# define the data loader
train_data = [(inputs[i], targets[i]) for i in range(len(inputs))]
train_loader = torch.utils.data.DataLoader(train_data, batch_size=1, shuffle=True)
print(train_loader.dataset)
# train the model
num_epochs = 10
train(model, w2v_model, optimizer, criterion, train_loader, num_epochs)

14
[('this is the input.', 'this is the target output.'), ('this is another input.', 'this is another target output.')]


As we can see, the model is able to come up with the sentence ‘good i am fine ‘ if we feed it with the words ‘good’, achieving what we intended for it to do!